Download http://download.tensorflow.org/example_images/flower_photos.tgz

> tar -xvzf flower_photos.tgz

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from time import time
from PIL import Image
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

# Cargar imagenes

In [ ]:
path = '/home/rhermoza/data/flower_photos/'
img_size = 224
batch_size = 64

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(path, shuffle=False,
    target_size=(img_size, img_size), batch_size=batch_size)

In [ ]:
train_generator.class_indices

In [ ]:
random_img = np.random.choice(train_generator.filenames)
print(random_img)
Image.open(path + random_img)

In [ ]:
batch_x, batch_y = train_generator.next()
batch_x.shape, batch_y.shape

# Cargar la red entrenada

In [ ]:
model = ResNet50(include_top=False)
model.summary()

In [ ]:
model.input, model.output

In [ ]:
t0 = time()
output = model.predict_on_batch(batch_x)
elapsed = time() - t0
output.shape, elapsed

In [ ]:
print('%.2f minutos por epoch.' % ((train_generator.samples // batch_size) * elapsed / 60))

# Preprocesar las imagenes con la red entrenada

In [ ]:
n_batch = train_generator.samples // batch_size
n_batch

In [ ]:
train_generator.reset()
outputs_x = []
outputs_y = []

for i in range(n_batch):
    print('%d/%d...' % (i+1, n_batch), end='\r')
    batch_x, batch_y = train_generator.next()
    outputs_x.append(model.predict_on_batch(batch_x))
    outputs_y.append(batch_y)

In [ ]:
outputs_x = np.vstack(outputs_x)
outputs_y = np.vstack(outputs_y)
outputs_x.shape, outputs_y.shape

In [ ]:
# Redimensionando
outputs_x = outputs_x[:,0,0,:]
outputs_x.shape, outputs_y.shape

In [ ]:
outputs_x[0], outputs_y[0]

In [ ]:
# # Guardar arrays
# np.savez_compressed('3-outputs', x=outputs_x, y=outputs_y)

In [ ]:
# Cargar arrays
outputs_x = np.load('3-outputs.npz')['x']
outputs_y = np.load('3-outputs.npz')['y']

In [ ]:
outputs_x.shape, outputs_y.shape

# Entrenar a partir de los features extraidos

In [ ]:
flowers_model = Sequential()
flowers_model.add(Dense(128, activation='relu', input_shape=(2048,)))
flowers_model.add(Dense(5, activation='softmax'))
flowers_model.compile(loss='categorical_crossentropy',
                      optimizer=Adam(0.001),
                      metrics=['accuracy'])
flowers_model.summary()

In [ ]:
log = flowers_model.fit(outputs_x, outputs_y, epochs=5, batch_size=256)

In [ ]:
plt.plot(log.history['loss'], '-o');

# Definiendo el modelo completo

In [ ]:
base_model = ResNet50(include_top=False)

In [ ]:
final_model = Sequential()
final_model.add(base_model)
final_model.add(flowers_model)
final_model.summary()

# Usando el modelo

In [ ]:
random_img = np.random.choice(train_generator.filenames)
print(random_img)
img = Image.open(path + random_img)
img

In [ ]:
# TODO:
# resize img
# pasar a formato de array
# redimensionar
# preprocesar

In [ ]:
# Obtener prediccion

In [ ]:
# Se crea un diccionario de indice a clase
index2class = {train_generator.class_indices[k]:k for k in train_generator.class_indices}
index2class

In [ ]:
# Usando el diccionario y la funcion np.argmax, obtener el nombre de la clase predecida
index2class[np.argmax()]

# Resumiendo en una funcion

In [ ]:
def get_class(path):
    # TODO
    return pred

In [ ]:
get_class(np.random.choice(train_generator.filenames))